In [ ]:
# def query_embedding(embeddings_dict, case_number, patch_number):
#     if case_number < 10:
#         case_number='0'+str(case_number)
#     if patch_number < 10:
#         patch_number='0'+str(patch_number)
#     query = 'EXACT09_CASE'+str(case_number)+'_'+str(patch_number)+'.nii.gz'
#     # print(query)
#     return embeddings_dict[query],query


# # query_emb1= query_embedding(embeddings_dict,2, 347).flatten()
# # query_emb2= query_embedding(embeddings_dict,2,467).flatten()
# # query_emb3= query_embedding(embeddings_dict,11, 165).flatten()
# # query_emb2.shape

# #第一组
# # input_cu_pairs = [(7, 403),(15, 347),(1, 539),(15, 355),(17, 347),(1, 411),(9, 403),(18, 419),(12, 227),(12, 218)]
# # input_xi_pairs= [(11, 165),(16, 228),(11, 153),(10, 153),(19, 89),(19, 90),(8, 154),(14, 281),(14, 285),(9, 154)]
# # input_wu_pairs= [(13, 46),(13, 166),(12, 151),(8, 277),(7, 143),(12, 169),(10, 168),(8, 160),(10, 167),(20, 278)]
# # triple_pairs_dict1={'cu':input_cu_pairs,'xi':input_xi_pairs,'wu':input_wu_pairs}

# input_cu_pairs = [(7, 403),(15, 347),(1, 539),(15, 355),(17, 347),(1, 411),(9, 403),(18, 419),(12, 227),(12, 218)]
# input_xi_pairs= [(11, 165),(16, 228),(11, 153),(10, 153),(19, 89),(19, 90),(8, 154),(14, 281),(14, 285),(9, 154)]
# input_wu_pairs= [(13, 46),(13, 166),(12, 151),(8, 277),(7, 143),(12, 169),(10, 168),(8, 160),(10, 167),(20, 278)]
# triple_pairs_dict1={'cu':input_cu_pairs,'xi':input_xi_pairs,'wu':input_wu_pairs}


# # 生成查询并存储在列表中
# query_list1 = []
# query_list2 = []
# query_list3 = []
# temp_dict=triple_pairs_dict1
# input_cu_pairs = temp_dict['cu']
# input_xi_pairs= temp_dict['xi']
# input_wu_pairs= temp_dict['wu']
# emb_vector,_=query_embedding(embeddings_dict, input_xi_pairs[1][0], input_xi_pairs[1][1])
# emb_vectors1=np.zeros((10, *emb_vector.shape))
# emb_vectors2=np.zeros((10, *emb_vector.shape))
# emb_vectors3=np.zeros((10, *emb_vector.shape))

# for i in range(10):
#     emb_vector1, query1 = query_embedding(embeddings_dict, input_cu_pairs[i][0], input_cu_pairs[i][1])
#     emb_vectors1[i,:]=emb_vector1
#     query_list1.append(query1)
#     emb_vector2, query2 = query_embedding(embeddings_dict, input_xi_pairs[i][0], input_xi_pairs[i][1])
#     emb_vectors2[i,:]=emb_vector2
#     query_list2.append(query2)
#     # emb_vector3, query3 = query_embedding(embeddings_dict, input_wu_pairs[i][0], input_wu_pairs[i][1])
#     # emb_vectors3[i,:]=emb_vector3
#     # query_list3.append(query3)


In [1]:
import numpy as np
import torch
import os
import skimage.io as io
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch import from_numpy as from_numpy
from matplotlib.colors import ListedColormap
from active_learning_utils import process_images
import pickle

import sys
sys.path.append('../')  # 将上一层目录添加到模块搜索路径中
from func.model_arch2 import SegAirwayModel

/home/wangc/anaconda3/envs/test1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

Precrop_dataset_for_train_path = "/mnt/wangc/EXACT09/Precrop_dataset_for_EXACT09_256"
Precrop_dataset_for_train_raw_path = Precrop_dataset_for_train_path+"/image"
Precrop_dataset_for_train_label_path = Precrop_dataset_for_train_path+"/label"

raw_case_name_list = os.listdir(Precrop_dataset_for_train_raw_path)
print(len(raw_case_name_list))

lidc_dataset_for_train_path='/mnt/wangc/LIDC/Precrop_dataset_for_LIDC-IDRI_256'
lidc_dataset_for_train_raw_path =lidc_dataset_for_train_path+"/image"
lidc_dataset_for_train_label_path = lidc_dataset_for_train_path+"/label"
lidc_raw_case_name_list = os.listdir(lidc_dataset_for_train_raw_path)
print(len(lidc_raw_case_name_list))

1184
2816


In [3]:
# 从文件加载
# file_path = '/home/wangc/now/NaviAirway/saved_var/subset_no_0304_postive.pkl'
file_path1='/home/wangc/now/NaviAirway/saved_var/exact09_256_embeddings_data.pkl'
file_path2='/home/wangc/now/NaviAirway/saved_var/lidc_256_embeddings_data.pkl'
# file_path = '/home/wangc/now/NaviAirway/saved_var/num_8832_embeddings_data.pkl'
with open(file_path1, 'rb') as file:
    loaded_data = pickle.load(file)
    exact_embeddings_list = loaded_data['embeddings_list']
    exact_embeddings_dict = loaded_data['embeddings_dict']
exact_stacked_embeddings_numpy = np.stack(exact_embeddings_list, axis=0)

with open(file_path2, 'rb') as file:
    loaded_data = pickle.load(file)
    lidc_embeddings_list = loaded_data['embeddings_list']
    lidc_embeddings_dict = loaded_data['embeddings_dict']
lidc_stacked_embeddings_numpy = np.stack(lidc_embeddings_list, axis=0)

# exact_lidc_concatenated_array = np.concatenate((exact_stacked_embeddings_numpy, lidc_stacked_embeddings_numpy), axis=0)
# merged_dict={**exact_embeddings_dict,**lidc_embeddings_dict}
# merged_list=raw_case_name_list+lidc_raw_case_name_list
# exact_stacked_embeddings_numpy.shape,lidc_stacked_embeddings_numpy.shape,exact_lidc_concatenated_array.shape


In [4]:
exact_lidc_concatenated_array = np.concatenate((exact_stacked_embeddings_numpy, lidc_stacked_embeddings_numpy), axis=0)
merged_dict={**exact_embeddings_dict,**lidc_embeddings_dict}
merged_list=raw_case_name_list+lidc_raw_case_name_list
exact_stacked_embeddings_numpy.shape,lidc_stacked_embeddings_numpy.shape,exact_lidc_concatenated_array.shape

((1184, 1, 256, 4, 16, 16),
 (2816, 1, 256, 4, 16, 16),
 (4000, 1, 256, 4, 16, 16))

In [21]:
xi=[('e', 3, 17),
 ('e', 2, 7),
 ('e', 1, 24),
 ('e', 4, 21),
 ('e', 8, 8),
 ('l', 66, 36),
 ('l', 328,50),
 ('l', 328, 22),
 ('l', 376, 22),
 ('l', 722, 26)]



cu=[('e', 1, 52),
 ('e', 2, 55),
 ('e', 2, 70),
 ('e', 3, 77),
 ('l', 140, 55),
 ('l', 403, 21),
 ('l', 698, 34),
 ('l', 744, 46),
 ('l', 403, 37),
 ('l', 673, 69)]

In [10]:
cu[0][2]

52

In [17]:
merged_dict.keys()

dict_keys(['EXACT09_CASE01_0.nii.gz', 'EXACT09_CASE01_1.nii.gz', 'EXACT09_CASE01_2.nii.gz', 'EXACT09_CASE01_3.nii.gz', 'EXACT09_CASE01_4.nii.gz', 'EXACT09_CASE01_5.nii.gz', 'EXACT09_CASE01_6.nii.gz', 'EXACT09_CASE01_7.nii.gz', 'EXACT09_CASE01_8.nii.gz', 'EXACT09_CASE01_9.nii.gz', 'EXACT09_CASE01_10.nii.gz', 'EXACT09_CASE01_11.nii.gz', 'EXACT09_CASE01_12.nii.gz', 'EXACT09_CASE01_13.nii.gz', 'EXACT09_CASE01_14.nii.gz', 'EXACT09_CASE01_15.nii.gz', 'EXACT09_CASE01_16.nii.gz', 'EXACT09_CASE01_17.nii.gz', 'EXACT09_CASE01_18.nii.gz', 'EXACT09_CASE01_19.nii.gz', 'EXACT09_CASE01_20.nii.gz', 'EXACT09_CASE01_21.nii.gz', 'EXACT09_CASE01_22.nii.gz', 'EXACT09_CASE01_23.nii.gz', 'EXACT09_CASE01_24.nii.gz', 'EXACT09_CASE01_25.nii.gz', 'EXACT09_CASE01_26.nii.gz', 'EXACT09_CASE01_27.nii.gz', 'EXACT09_CASE01_28.nii.gz', 'EXACT09_CASE01_29.nii.gz', 'EXACT09_CASE01_30.nii.gz', 'EXACT09_CASE01_31.nii.gz', 'EXACT09_CASE01_32.nii.gz', 'EXACT09_CASE01_33.nii.gz', 'EXACT09_CASE01_34.nii.gz', 'EXACT09_CASE01_35.

In [28]:
def query_embedding(embeddings_dict, prefix,case_number, patch_number):
    if case_number < 10:
        case_number='0'+str(case_number)
    elif 20<case_number<100:
        case_number='0'+str(case_number)
    # if patch_number < 10:
    #     patch_number='0'+str(patch_number)
    query = prefix+str(case_number)+'_'+str(patch_number)+'.nii.gz'
    # print(query)
    return embeddings_dict[query],query
    # return query
for three_tuple in xi:
    query_index={'e':'EXACT09_CASE','l':'LIDC_IDRI_0'}
    # print(three_tuple)
    if three_tuple[0] == 'e':
        vector,_=query_embedding(merged_dict,query_index['e'],three_tuple[1],three_tuple[2])
    if three_tuple[0] == 'l':
        vector,_=query_embedding(merged_dict,query_index['l'],three_tuple[1],three_tuple[2])


In [32]:

# 生成查询并存储在列表中
query_list1 = []
query_list2 = []
query_list3 = []


# for three_tuple in xi:
#     query_index={'e':'EXACT09_CASE','l':'LIDC_IDRI_0'}
#     # print(three_tuple)
#     if three_tuple[0] == 'e':
#         vector,_=query_embedding(query_index['e'],merged_dict,three_tuple[1],three_tuple[2])
#     if three_tuple[0] == 'l':
#         vector,_=query_embedding(query_index['l'],merged_dict,three_tuple[1],three_tuple[2])

three_tuple1=xi[0]    
if three_tuple[0] == 'e':
    emb_vector,_=query_embedding(merged_dict,query_index['e'],three_tuple[1],three_tuple[2])
if three_tuple[0] == 'l':
    emb_vector,_=query_embedding(merged_dict,query_index['l'],three_tuple[1],three_tuple[2])
emb_vectors1=np.zeros((10, *emb_vector.shape))
emb_vectors2=np.zeros((10, *emb_vector.shape))
# emb_vectors3=np.zeros((10, *emb_vector.shape))

for i in range(10):
    query_index={'e':'EXACT09_CASE','l':'LIDC_IDRI_0'}
    # print(three_tuple)
    # if three_tuple[0] == 'e':

        # emb_vector1, query1 = query_embedding(merged_dict, query_index[xi[i][0]],xi[i][1], xi[i][2])
        # emb_vector2, query2 = query_embedding(merged_dict, query_index[cu[i][0]],cu[i][1], cu[i][2])
    # if three_tuple[0] == 'l':
    emb_vector1, query1 = query_embedding(merged_dict, query_index[xi[i][0]],xi[i][1], xi[i][2])
    emb_vector2, query2 = query_embedding(merged_dict, query_index[cu[i][0]],cu[i][1], cu[i][2])

    emb_vectors1[i,:]=emb_vector1
    query_list1.append(query1)

    emb_vectors2[i,:]=emb_vector2
    query_list2.append(query2)
    
    # emb_vector3, query3 = query_embedding(embeddings_dict, input_wu_pairs[i][0], input_wu_pairs[i][1])
    # emb_vectors3[i,:]=emb_vector3
    # query_list3.append(query3)

In [33]:
def cul_trinity(emb_vectors,emb_vectors1=emb_vectors1,emb_vectors2=emb_vectors2,emb_vectors3=emb_vectors3):
    similarity1 = similarity_sum(emb_vectors, emb_vectors1)
    similarity2 = similarity_sum(emb_vectors, emb_vectors2)
    similarity3 = similarity_sum(emb_vectors, emb_vectors3)
    return similarity1,similarity2,similarity3

similarity1,similarity2,similarity3=cul_trinity(class0_mat,emb_vectors1=emb_vectors1,emb_vectors2=emb_vectors2,emb_vectors3=emb_vectors3)
similarity1,similarity2,similarity3

import numpy as np

def get_mv_and_cm(emb_vectors):
    # mv: Mean Vector, cm: Covariation Matrix
    vector_num = emb_vectors.shape[0]
    # 初始化协方差矩阵
    covariation_matrix = np.zeros((vector_num, vector_num))
    # 将向量展开为二维数组 (vector_num, feature_dim)
    vectors = emb_vectors.reshape(vector_num, -1)
    # 计算均值向量
    mean_vector = np.mean(vectors, axis=0)
    # 中心化向量
    centered_vectors = vectors - mean_vector
    
    # 计算协方差矩阵
    for i in range(vector_num):
        for j in range(vector_num):
            covariation_matrix[i, j] = np.dot(centered_vectors[i], centered_vectors[j]) / (vector_num - 1)
    
    return mean_vector, covariation_matrix



# def get_mahalanobis_distance(vector,mean_vector,covariation_matrix):
#     vector=vector.flatten()
#     temp=vector-mean_vector


import os
import matplotlib.pyplot as plt
from skimage import io
from matplotlib import gridspec

def cul_background(label_path,slice_index=None):
    label_img_list = os.listdir(label_path)
    count=0
    i=0
    for name in label_img_list:
        i+=1
        label_img_addr = os.path.join(label_path, name)  
        label_img = io.imread(label_img_addr, plugin='simpleitk')
        if slice_index is None:
            if 1 not in label_img:
                count+=1
        else:
            if 1 not in label_img[slice_index, :, :]:
                count+=1
    return count/i

        
   

['EXACT09_CASE03_17.nii.gz',
 'EXACT09_CASE02_7.nii.gz',
 'EXACT09_CASE01_24.nii.gz',
 'EXACT09_CASE04_21.nii.gz',
 'EXACT09_CASE08_8.nii.gz',
 'LIDC_IDRI_0066_36.nii.gz',
 'LIDC_IDRI_0328_50.nii.gz',
 'LIDC_IDRI_0328_22.nii.gz',
 'LIDC_IDRI_0376_22.nii.gz',
 'LIDC_IDRI_0722_26.nii.gz']